In [3]:
import numpy as np
import pandas as pd

def mm_chain_order(p=None):
    '''
    :param p: Sequence of shape such that the ith matrix has shape = (p[i-1], p[i])
    :return: The minimum number of scalar mulptiply operations
    '''
    if p is None:
        raise Exception('Unvalid p')
    # n: number of matrices in sequence
    n = len(p) - 1
    # table saving optimal values of subproblems
    cost_table = pd.DataFrame(columns=range(1, n + 1), index=range(1, n + 1))
    # table saving index of the subproblems
    strack_table = cost_table.copy()
    # table saving how to multiply the matrices, order in parenthesizating
    solution_table = strack_table.copy()
    
    # sub-sequence of matrix has 1 of length -> no any multiplication
    for i in range(1,n+1):
        cost_table[i][i] = 0
        
    # sub-sequence of matrix must have at least 2, at most n matrices
    for sub_len in range(2,n+1):
        # all posiply sub-sequence of matrices with particular length
        for sub_start in range(1,n-sub_len+2):
            sub_end = sub_start + sub_len - 1
            # set cost value ∞ first for any sub-sequence 
            cost_table[sub_end][sub_start] = np.inf
            # split particular sub-sequence of matrices into 2 sub-sequence A[sub_start,k] and A[k+1,sub_end] (inclusive)
            # spliting means parenthesizating (A[sub_start,k]) and (A[k+1,sub_end])
            for k in range(sub_start,sub_end):
                # compute cost value for each spliting
                cur_cost = cost_table[k][sub_start] + cost_table[sub_end][k+1] + p[sub_start-1]*p[k]*p[sub_end]
                # check current whether is min cost
                if cur_cost <  cost_table[sub_end][sub_start]:
                    # update the min cost for the sub-sequence
                    cost_table[sub_end][sub_start] = cur_cost
                    # store k (index where is cut in sub-sequence) of the sub-sequence such that cost value is min
                    strack_table[sub_end][sub_start] = k

    for i in strack_table.index:
        for j in strack_table.columns:
            if i >= j:
                continue
            # construct solution (how to parenthesize) sequence of matrices A[i -> j]
            solution_table[j][i] = construct_solution(strack_table,i,j)
    return cost_table,solution_table


def construct_solution(strack_table,i,j):
    if i == j:
        solution = "A" + str(int(i))
    else:
        solution = '('
        solution += construct_solution(strack_table,i,strack_table[j][i])
        solution += construct_solution(strack_table, strack_table[j][i]+1,j)
        solution += ')'
    return solution

docstring = '''
    Compact sequence of shapes of n matrices is sequence forms:
    (p0,p1,p2,...,pn)
    The ith matrix A has shape (p[i-1],p[i]), i = 1,2,3,4,...,n
    Enter compact sequence: 
'''
compact_sequence_of_shape = [int(i) for i in input(docstring).split(' ')]
cost,solution = mm_chain_order(compact_sequence_of_shape)
print(cost)
solution
#Input: 30 35 35 15 15 5 5 10 10 20 20 25
# 30 35 15 5 10 20 25
#Output: 15125  ((A1(A2A3))((A4 A5)A6))


    Compact sequence of shapes of n matrices is sequence forms:
    (p0,p1,p2,...,pn)
    The ith matrix A has shape (p[i-1],p[i]), i = 1,2,3,4,...,n
    Enter compact sequence: 
30 35 15 5 10 20 25
     1      2     3     4      5      6
1    0  15750  7875  9375  11875  15125
2  NaN      0  2625  4375   7125  10500
3  NaN    NaN     0   750   2500   5375
4  NaN    NaN   NaN     0   1000   3500
5  NaN    NaN   NaN   NaN      0   5000
6  NaN    NaN   NaN   NaN    NaN      0


,1,2,3,4,5,6
1,NaN,(A1A2),(A1(A2A3)),((A1(A2A3))A4),((A1(A2A3))(A4A5)),((A1(A2A3))((A4A5)A6))
2,NaN,NaN,(A2A3),((A2A3)A4),((A2A3)(A4A5)),((A2A3)((A4A5)A6))
3,NaN,NaN,NaN,(A3A4),(A3(A4A5)),(A3((A4A5)A6))
4,NaN,NaN,NaN,NaN,(A4A5),((A4A5)A6)
5,NaN,NaN,NaN,NaN,NaN,(A5A6)
6,NaN,NaN,NaN,NaN,NaN,NaN
